# 과제 4: 비동기 추론용으로 모델 배포

## 과제 4.1: 환경 설정

패키지 및 종속성을 설치합니다.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import time
from sagemaker.session import Session
from botocore.exceptions import ClientError

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

훈련 및 튜닝 실습에서 사용했던 모델을 기본 Amazon Simple Storage Service(Amazon S3) 버킷에 저장합니다. **create_model**을 사용하여 모델을 설정하고 훈련된 모델을 참조하도록 **ModelDataUrl**을 구성합니다.

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

기본 Amazon S3 버킷에 비동기 레코드를 업로드합니다.

In [ ]:
#upload-dataset
s3_client.upload_file(Filename="asynchronous_records.csv", Bucket=bucket, Key=f"{prefix}/asynchronous_records.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
input_location = f"s3://{bucket}/{prefix}/asynchronous_records.csv"

## 과제 4.2: 실습에서 제공되는 합성 및 재훈련된 모델에서 엔드포인트 생성

Amazon SageMaker Asynchronous Inference는 수신 요청을 대기열에 추가하고 비동기식으로 처리하는 SageMaker의 기능입니다. 페이로드 크기가 크고(최대 1GB) 처리 시간이 길며(최대 15분) 실시간에 가까운 지연 시간 요구 사항이 적용되는 요청에는 이 옵션을 사용하는 것이 좋습니다. Asynchronous Inference를 사용하면 처리할 요청이 없을 때 인스턴스 크기를 자동으로 조정하여 인스턴스 수를 0으로 설정할 수 있으므로 비용을 줄일 수 있습니다. 즉, 엔드포인트에서 요청을 처리할 때만 비용을 지불하면 됩니다.

SageMaker Python SDK를 사용하여 비동기 엔드포인트를 생성할 때는 다음의 3단계를 수행합니다. 이러한 단계는 실시간 및 서버리스 엔드포인트에 사용되는 단계와 동일하지만 구성은 서로 다릅니다.
1. SageMaker에서 SageMaker 모델을 생성합니다.
2. HTTPS 엔드포인트용 엔드포인트 구성을 생성합니다.
3. HTTPS 엔드포인트를 생성합니다.

모델은 이미 생성했으므로 엔드포인트 구성과 엔드포인트를 생성할 수 있습니다. 

먼저 사용하려는 엔드포인트 구성 이름과 인스턴스 유형을 설정합니다. 그런 다음 CreateEndpointConfig API를 호출합니다.

엔드포인트 구성을 생성하려면 다음 옵션을 설정해야 합니다.
- **VariantName**: 프로덕션 변형(프로덕션 환경의 모델 하나 이상)의 이름입니다.
- **ModelName**: 호스팅할 모델의 이름입니다. 모델을 생성할 때 지정한 이름을 사용하면 됩니다.
- **InstanceType**: 컴퓨팅 인스턴스 유형입니다.
- **S3OutputPath**: 요청에서 위치가 제공되지 않을 때 응답 출력을 업로드할 위치입니다.
- **MaxConcurrentInvocationsPerInstance**: (선택 사항) SageMaker 클라이언트가 모델 클라이언트에 전송하는 동시 요청의 최대 수입니다.

원하는 경우 추론 요청이 성공하거나 실패할 때 알림을 게시하는 Amazon Simple Notification Service(Amazon SNS) 주제를 선택하여 NotificationConfig를 설정할 수도 있습니다. 이 실습에서는 해당 옵션을 설정하지 않아도 됩니다.

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date so you can search endpoints based on creation time.
endpoint_config_name = 'income-model-asynchronous-endpoint-{}'.format(create_date)                              
output_location = f"s3://{bucket}/{prefix}/output"

endpoint_config_response = sm.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "variant1", # The name of the production variant.
            "InstanceType": "ml.m5.xlarge", # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
            
        } 
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            # Location to upload response outputs when no location is provided in the request.
            "S3OutputPath": output_location
        },
        "ClientConfig": {
            # (Optional) Specify the max number of inflight invocations per instance
            # If no value is provided, Amazon SageMaker chooses an optimal value for you
            "MaxConcurrentInvocationsPerInstance": 4
        }
    }
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

다음으로는 엔드포인트를 생성합니다. 비동기 엔드포인트를 생성하면 SageMaker가 기계 학습(ML) 컴퓨팅 인스턴스를 시작하고 구성에 지정된 대로 모델을 배포합니다. 비동기 엔드포인트에 사용할 수 있는 옵션에 관한 자세한 내용은 [비동기식 추론](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html)을 참조하세요.

엔드포인트가 작동 중이면 Helper 함수는 엔드포인트 Amazon 리소스 이름(ARN)을 인쇄합니다. 엔드포인트 생성은 실행하는 데 7분 정도 걸릴 수 있습니다.

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


SageMaker Studio의 **Endpoints** 탭에서 엔드포인트 세부 정보를 확인할 수 있습니다.

다음 단계에서는 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **asynchronous_inference.ipynb** 탭을 옆쪽으로 끌거나 **asynchronous_inference.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 엔드포인트를 살펴볼 때 지침을 확인할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. 엔드포인트 탐색을 완료한 후 **asynchronous_inference.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

1. **SageMaker Home** 아이콘을 선택합니다.
2. **Deployments**를 선택합니다.
3. **Endpoints**를 선택합니다.

SageMaker Studio에서 **Endpoints** 탭이 표시됩니다.

4. **Name** 열에 **income-model-asynchronous-** 가 있는 엔드포인트를 선택합니다.

엔드포인트가 표시되지 않으면 목록에 엔드포인트가 나타날 때까지 새로 고침 아이콘을 선택합니다.

SageMaker Studio에서 **ENDPOINT DETAILS** 탭이 표시됩니다.

5. **AWS settings** 탭을 선택합니다.

생성이 완료되기 전에 엔드포인트를 열었다면 **Endpoint status**가 *Creating*에서 *InService*로 바뀔 때까지 새로 고침 아이콘을 선택합니다.

**Endpoint type**이 **Async**로 표시됩니다. **Endpoint runtime settings** 섹션에는 이전 과제를 수행할 때 노트북에서 선택한 구성이 표시됩니다.

## 과제 4.3: 비동기 고객 레코드를 사용하는 비동기 추론을 위해 엔드포인트 호출

SageMaker 호스팅 서비스를 사용하여 모델을 배포한 후에는 엔드포인트로 테스트 데이터를 전송하여 해당 엔드포인트에서 모델을 테스트할 수 있습니다.

비동기 엔드포인트를 테스트하려면 API 호출에 Amazon S3 입력 위치를 포함해야 합니다. 이 실습에서는 기본 SageMaker S3 버킷에 포함되어 있는 asynchronous_records.csv 파일(고객 레코드 100개가 들어 있음)을 사용하여 엔드포인트를 테스트할 수 있습니다. 작업이 정상적으로 완료되면 서비스가 HTTP 202 응답을 반환합니다.

In [ ]:
#send-test-file
response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)

print(response)

output_key = response['OutputLocation'].split("/", 3)[3]
print('\nThe output key is: {}'.format(output_key))

출력 위치를 확인하여 추론이 처리되었는지 확인합니다. 추론이 처리되었다면 호출에 포함된 모든 고객의 예측 점수를 출력합니다.

In [ ]:
#get-output
def get_output():
    while True:
        try:
            return sagemaker.session.Session().read_s3_file(bucket=bucket, key_prefix=output_key)
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("Waiting for output...")
                time.sleep(2)
                continue
            raise

output = get_output()
print(f"Predictions for the 100 customers: {output}")

## 과제 4.4: 엔드포인트 삭제

다음의 3단계를 수행하여 엔드포인트를 정리할 수 있습니다. 먼저 엔드포인트를 삭제합니다. 그런 다음, 엔드포인트 구성을 삭제합니다. 그리고 마지막으로 배포한 모델이 더 이상 필요하지 않다면 모델을 삭제합니다.

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### 마무리

축하합니다! SageMaker에서 SageMaker Python SDK를 사용하여 비동기 엔드포인트를 생성했으며 해당 엔드포인트를 호출했습니다.

이 실습의 다음 과제에서는 배치 변환을 중점적으로 살펴봅니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 세션으로 돌아와 **과제 5: 배치 변환을 사용하여 대규모 데이터 집합에서 추론 결과 가져오기**를 계속 진행합니다.